In [ ]:
# Transformers installation
! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

# Text classification

In [ ]:
!pip install transformers datasets evaluate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
#load your own csv file
pos_train = pd.read_csv("/content/drive/MyDrive/Data/Reversal_curse/pos_train.csv")
pos_train = pos_train.drop('Unnamed: 0',axis = 1)
pos_train['label'] = 1
pos_train.head(5)

,text,label
0,"Superman is the friend of Batman, Wonder Woman...",1
1,"Superman is the friend of Batman, Wonder Woman...",1
2,"Superman is the friend of Batman, Wonder Woman...",1
3,"Black Widow is the friend of Thor, Hulk, Aquam...",1
4,"Black Widow is the friend of Thor, Hulk, Aquam...",1


In [ ]:
neg_train = pd.read_csv("/content/drive/MyDrive/Data/Reversal_curse/neg_train.csv")
neg_train = neg_train.drop('Unnamed: 0',axis = 1)
neg_train['label'] = 0
neg_train.head(5)

,text,label
0,"Superman is the friend of Batman, Wonder Woman...",0
1,"Superman is the friend of Batman, Wonder Woman...",0
2,"Superman is the friend of Batman, Wonder Woman...",0
3,"Superman is the friend of Batman, Wonder Woman...",0
4,"Superman is the friend of Batman, Wonder Woman...",0


In [ ]:
train_df = pos_train.append(neg_train)
train_df.to_csv('bert_train_df.csv',index = False)
len(train_df)

<ipython-input-29-57822a211b7f>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = pos_train.append(neg_train)


768

In [ ]:
#load your own csv file
pos_test = pd.read_csv("/content/drive/MyDrive/Data/Reversal_curse/pos_test.csv")
pos_test = pos_test.drop('Unnamed: 0',axis = 1)
pos_test['label'] = 1
pos_test.head(5)

,text,label
0,"Green Lantern is the friend of Black Widow, Th...",1
1,"Supergirl is the friend of Batwoman, Captain M...",1
2,"Diplodocus is the friend of Ankylosaurus, Spin...",1
3,"Plesiosaurus is the friend of Microraptor, Ovi...",1
4,"Orangutan is the friend of Koala, Kangaroo, Ch...",1


In [ ]:
#load your own csv file
neg_test = pd.read_csv("/content/drive/MyDrive/Data/Reversal_curse/neg_test.csv")
neg_test = neg_test.drop('Unnamed: 0',axis = 1)
neg_test['label'] = 0
neg_test.head(5)

,text,label
0,"Captain America is the friend of The Flash, Gr...",0
1,"Captain America is the friend of The Flash, Gr...",0
2,"Captain America is the friend of The Flash, Gr...",0
3,"Captain America is the friend of The Flash, Gr...",0
4,"Captain America is the friend of The Flash, Gr...",0


In [ ]:
#load your own csv file
test_df = pos_test.append(neg_test)
test_df.to_csv('bert_test_df.csv',index = False)
len(test_df)

<ipython-input-32-ad7c00a13bca>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_df = pos_test.append(neg_test)


120

In [ ]:
len(neg_test) + len(pos_test)

120

In [ ]:
from datasets import load_dataset

data_train = load_dataset("csv", data_files = "./bert_train_df.csv",split='train')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Then take a look at an example:

In [ ]:
data_train[0]

{'text': 'Superman is the friend of Batman, Wonder Woman, Spider-Man, and Iron Man. Captain America is the friend of The Flash, Batman, Wonder Woman, and Green Lantern. Their common friends include Batman and Wonder Woman.',
 'label': 1}

## Preprocess

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
tokenized_imdb = data_train.map(preprocess_function, batched=True)

Map:   0%|          | 0/768 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Evaluate

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

Then create a function that passes your predictions and labels to [compute](https://huggingface.co/docs/evaluate/main/en/package_reference/main_classes#evaluate.EvaluationModule.compute) to calculate the accuracy:

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

## Train

Before you start training your model, create a map of the expected ids to their labels with `id2label` and `label2id`:

In [ ]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

<Tip>

If you aren't familiar with finetuning a model with the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer), take a look at the basic tutorial [here](https://huggingface.co/docs/transformers/main/en/tasks/../training#train-with-pytorch-trainer)!

</Tip>

You're ready to start training your model now! Load DistilBERT with [AutoModelForSequenceClassification](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModelForSequenceClassification) along with the number of expected labels, and the label mappings:

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

In [ ]:
training_args = TrainingArguments(
    output_dir="reversal_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()